In [18]:
from openai import OpenAI

In [5]:
from datasets import load_dataset

flights = load_dataset("nuprl/engineering-llm-systems", name="flights", split="train")


README.md:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/467k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31601 [00:00<?, ? examples/s]

In [6]:
import datetime
from typing import List, Optional

class Flight:
    def __init__(self, flight_id: int):
        self.flight_info = flights[flight_id]

def find_flights(origin: str, destination: str, date: datetime.date) -> List[Flight]:
    formatted_date = date.strftime("%Y-%m-%d")
    return [Flight(flight["id"]) for flight in flights if flight["origin"] == origin and flight["destination"] == destination and flight["date"] == formatted_date]

def book_flight(flight_id: int) -> Optional[int]:
    if flights[flight_id]["available_seats"] > 0:
        return flight_id
    else: 
        return None


In [17]:
def wrap(text):
    """
    Some basic text wrapping code that works better for me than the built-in
    textwrap module.
    """
    result_lines = [ ]
    current_line = []
    current_line_length = 0
    for line in text.split("\n"):
        for words in line.split(" "):
            if current_line_length + len(words) > 80:
                result_lines.append(" ".join(current_line))
                current_line = [ ]
                current_line_length = 0
            current_line.append(words)
            current_line_length += len(words) + 1
        result_lines.append(" ".join(current_line))
        current_line = [ ]
        current_line_length = 0
    return "\n".join(result_lines)

In [19]:
API_KEY = "currier.l@northeastern.edu:19284"

In [ ]:
def run_chat(api_key: str):
    base_url = "https://nerc.guha-anderson.com/v1"
    
    codegen_prompt = """
    You are an airline chatbot. Only answer airline queries. We have defined a function called def 
    find_flights(origin: str, destination: str, departure_date: datetime.date) -> list[dict]:

    It takes the origin and destination airport codes and produces a list of dictionaries containing flight information. 
    
    Stuff like this:
    { 'id': 407,
    'date': datetime.date(2023, 1, 6),
    'airline': 'Delta',
    'flight_number': 'DL9926',
    'origin': 'ORD',
    'destination': 'JFK',
    'departure_time': datetime.time(20, 55),
    'arrival_time': datetime.time(22, 55),
    'available_seats': 172}

    If the user query is requesting information that you could find via this function, return the code you would use to find that information in this format: 
    '''
    { code }
    '''
    """

    textgen_prompt = """
    You are an airline chatbot. Only answer airline queries. We have defined a function called def 
    find_flights(origin: str, destination: str, departure_date: datetime.date) -> list[dict]:

    It takes the origin and destination airport codes and produces a list of dictionaries containing flight information. 
    
    Stuff like this:
    { 'id': 407,
    'date': datetime.date(2023, 1, 6),
    'airline': 'Delta',
    'flight_number': 'DL9926',
    'origin': 'ORD',
    'destination': 'JFK',
    'departure_time': datetime.time(20, 55),
    'arrival_time': datetime.time(22, 55),
    'available_seats': 172}

    If the user query is requesting information that you could find
    """

    client = OpenAI(base_url=base_url, api_key=api_key)

    while True: 
        user_input = input("User (blank to quit):")

        if user_input == "":
            print("[]")
            break
        else:
            print("User (blank to quit): "+ user_input)

        # chatbot logic here
        try:
            resp = client.chat.completions.create(
                messages = [{ "role": "system", "content": codegen_prompt },
                            { "role": "user", "content": "What flights are there from Boston to San Francisco on January 6, 2023? "},
                            { "role": "assistant", "content": """find_flights("BOS", "SFO", datetime.date(2023, 1, 6))""" },
                            { "role": "user", "content": user_input }],
                model = "llama3p1-8b-instruct",
                temperature=0)
            print(resp.choices[0].message.content)
        except:
            print("API key invalid, please try again.")
            break

run_chat(api_key = API_KEY)

[]
